In [1]:
import numpy
import matplotlib
import math
get_ipython().run_line_magic('matplotlib', 'inline')

def vel(Vdot, A):
    """
    Arguments
        Vdot - Volumetric Flow Rate of the Fluid
        A - Crossectional Area of the Pipe
        **Note - Constant Dimensions Required for ALL functions
    
    Output
        Returns the Velocity of the Fluid 
    """
    v_fluid = Vdot/A
    return v_fluid

def re(v_fluid, L, nu):
    """
    Arguments
        v_fluid - velocity of the fluid
        L - Critical Length
        nu - kinematic viscosity of the fluid
        
    Output
        Returns a Reynolds Matrix
        re_n[0] is the reynolds number
        re_n[1] is the type of flow 
            1 is laminar 
            0 is turbulent
    """
    re_n = v_fluid * L/nu
    
    if re_n <= 5*10**5:
        laminar = 1
    else: 
        laminar = 0
    re_prop = [re_n, laminar]
    return re_prop 

def nusselt(re_n, pr_n):
    """
    Arguments
        re_n - reynolds number of the flow 
        pr_n - prandall's number
        
    Output
        returns the nusselt number of a laminar flow over a flat plate
    """
    x = 1/3
    nusselt_n = 0.664* math.pow(re_n, 0.5) * math.pow(pr_n, x)
    return nusselt_n

def r_cond(L, k, A):
    """
    Arguments
        L - Length of the conductive heat transfer
        k - thermal conductivity coefficient of the material 
        A - cross sectional area perpendicular to the flow of heat
        
    Output
        returns the conductive resistance 
    """
    r_conduction = L/(k*A)
    return r_conduction


def r_multi_fin(t, w, L, h, N, k, fin_surface):
    P = 2 * w + 2 * t
    A_c = t * w
    A_f = 2 * w * L + t * w
    A_b = (N-1) * t * w
    A_t = N * A_f + A_b
    m = ((h * P)/(k * A_c))**.5
    M = (h * P * k * A_c)**.5
    
    qf = M * ((math.sinh(m*L) + (h/(m*k)) * math.cosh(m*L)))/(math.cosh(m*L) + (h/m*k) * math.sinh(m*L))
    q_max = h * A_f
    etaF = qf/q_max
    etaO = 1 - (N*A_f)/A_t * (1 - etaF)
           
    R_fin = 1/(etaO * h * A_t)
    R_fin_inv = R_fin**-1
    R_inv_fin = fin_surface * R_fin_inv   
    
    return R_inv_fin

def pr(Cp, u, k):
    """
    Arguments
        Cp - specific heat of the material 
        u - dynamic viscosity 
        k - thermal conductivity of the material 
        
    Output
        returns the prandalls number 
    """
    Pr = Cp*u/k
    return Pr

def h(k, L, re_n, pr_n):
    """
    Arguments
        k - thermal conductivity of the fluid
        L - perpendicular length
        re_n - reynolds number
        pr_n - prandalls number
        
    Output
        returns the convection coefficient for laminar flow over a flat plate 
    """
    x = 1/3
    h = (k/L) * 0.664 * math.pow(re_n, 0.5) * math.pow(pr_n, x)
    return h

## Material and Chip Properties 
q = 130                           #Power of the Chip (W)
t_inf = 38                        #Temperature inside the Case as Specified by Intel (C)

L_paste = 50 * (10**-6)           #Length of the Paste (m)
k_paste = 8.9                     #Thermal Conductivity of Paste (W/m^2 K)
A_paste = 0.0405 * 0.03674        #Area of Paste (m^2)

k_6061 = 175                      #Thermal Conductivity of 6061 Aluminum
k_copper = 401                    #Thermal Conductivity of Copper
k_petg = 0.29                     #Thermal Conductivity of Water Cooling Tubes

Cp_air = 1007                     #Specific Heat of Air J/kg*K
nu_air = 17.02 * 10**-6           #Kinimatic Viscosity of Air m^2/s
mu_air = 19.18*10**-6             #Dynamic Viscosity of Air kg/m*s 
k_air = 26.62*10**-3              #Thermal Conductivity of Air (W/m^2 K)

Cp_water = 4182                   #Specific Heat of Water J/kg*K
nu_water = 0.0000006591           #Kinematic Viscosity of Water at 40C
mu_water = 0.000653               #Dynamic Viscosity of Water at 40C
k_water = 0.62856                 #Thermal Conductivity of Water at 40C

## Air Cooling Properties
L_core = 0.01728
A_core = 0.0375**2
t_fin = 0.0015
w_fin = 0.050
L_fin = 0.040
N_fin = 13
N_fin_surfaces = 4

V_dot_fan = 0.03912444 #82.9CFM
A_fan = math.pi * 0.25 * (.12**2 - 0.035**2)
u_fan = vel(Vdot = V_dot_fan, A = A_fan)

re_air = re(v_fluid = u_fan, L = w_fin, nu = nu_air)
Pr_air = pr(Cp = Cp_air, u = mu_air, k = k_air)
Nusselt_air = nusselt(re_n = re_air[0], pr_n = Pr_air)
h_air = h(k = k_air, L = w_fin, re_n = re_air[0], pr_n = Pr_air)
r_paste = r_cond(L = L_paste, k = k_paste, A = A_paste)
r_core = r_cond(L = L_core, k = k_6061, A = A_core)
r_fin = r_multi_fin(t = t_fin, w = w_fin, L = L_fin, h = h_air, N = N_fin, k = k_6061, fin_surface = N_fin_surfaces)
r_eq = r_paste + r_fin + r_core
t_dtc_chip = q * r_eq + t_inf
print('The Temperature of the Chip with Air Cooling is: ', t_dtc_chip)

## Water Cooling Properties
L_base_w = 0.001
A_base_w = 0.045 * 0.042
t_fin_w = 0.005
b_fin_w = 0.005
h_fin_w = 0.004
w_fin_w = 0.045
L_fin_w = 0.042
N_fin_w = 5
N_fin_surface_w = 4
SA_fin_w = L_fin_w * h_fin_w * (2*N_fin_surface_w)
r_id = 0.00635
r_od = 0.009525
L_pipe = 3 

R_rad_w = 25**-1 #250W/10K --> 25W/K --> inv(25) K/W

V_dot_pump_w = 1.38893* (10**(-4))
A_pipe_w = math.pi * 0.25 * 0.00635**2 ##Area of a 1/4" ID pipe
u_pipe_w = vel(Vdot = V_dot_pump_w, A = A_pipe_w)
re_water = re(v_fluid = u_pipe_w, L = L_fin_w, nu = nu_water)
Pr_water = pr(Cp = Cp_water, u = mu_water, k = k_water)
Nusselt_water = nusselt(re_n = re_water[0], pr_n = Pr_water)
h_water = h(k = k_water, L = L_fin_w, re_n = re_water[0], pr_n = Pr_water)
m_w = math.sqrt(2*h_water / (k_copper * t_fin_w))
L_c_w = L_fin_w + (t_fin_w / 2)
n_fin_w = math.tanh(m_w * L_c_w) / (m_w * L_c_w)
r_paste_w = r_cond(L = L_paste, k = k_paste, A = A_paste)
r_core_w = r_cond(L = L_base_w, k = k_copper, A = A_base_w)
r_fin_w = (h_water * (b_fin_w * L_fin_w * N_fin_surface_w + N_fin_w * n_fin_w * SA_fin_w))**(-1)
r_tube = math.log(r_od/r_id)/(2 * math.pi * L_pipe * k_petg)
r_eq_w = r_paste_w + r_core_w + r_fin_w + R_rad_w + r_tube
t_dtc_chip_w = q*r_eq_w + t_inf
print('\nThe Temperature of the Chip with Water Cooling is: ', t_dtc_chip_w)

The Temperature of the Chip with Air Cooling is:  67.3286557083139

The Temperature of the Chip with Water Cooling is:  59.63942461497843
